In [1]:
###################################################################################################
#
# Copyright (C) 2023 Maxim Integrated Products, Inc. All Rights Reserved.
#
# Maxim Integrated Products, Inc. Default Copyright Notice:
# https://www.maximintegrated.com/en/aboutus/legal/copyrights.html
#
###################################################################################################

import os
import pandas as pd
import sys

import math
import numpy as np
import torch

sys.path.append(os.path.join(os.getcwd(), '..'))

from datasets import smsmotordata


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_path = '/data2/ml/'

class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

args = Args(act_mode_8bit=False)

is_file_idx_available = True

# Call loader with _for_analysis suffix to retrieve file indexes
train_set, test_set = smsmotordata.smsmotordata_dur_0_5_overlap_0_75_dbu_ind_get_datasets_for_train_for_analysis((data_path, args), load_train=True, load_test=True)



Pickle file path: /data2/ml/SMSMotorData/processed/train_anomaly_label_False_ds_ratio_2_signal_dur_0.5_overlap_ratio_0.75_dc_removed_False_dbu_ind_True_ds_after_fft_False_norm_per_ch_False.pkl ...


Generating dataset pickle files from the raw data files...

Train dataset length: 20160


Pickle file path: /data2/ml/SMSMotorData/processed/test_anomaly_label_False_ds_ratio_2_signal_dur_0.5_overlap_ratio_0.75_dc_removed_False_dbu_ind_True_ds_after_fft_False_norm_per_ch_False.pkl ...


Generating dataset pickle files from the raw data files...

Test dataset length: 5040



In [3]:
sample_idx = 62
print(test_set[sample_idx][0].shape)
print(test_set[sample_idx][1].shape)
print(test_set[sample_idx][2])

torch.Size([3, 512])
(3, 512)
74714


In [4]:
sample_idx = 62
sample_test_signal, sample_test_lbl, sample_test_file_id = test_set[sample_idx]

print(type(sample_test_signal))
print(sample_test_signal.shape)

print(type(sample_test_lbl))
print(sample_test_lbl.shape)

print([sample_test_signal.min(), sample_test_signal.max()])

print(type(sample_test_file_id))
print(sample_test_file_id)

<class 'torch.Tensor'>
torch.Size([3, 512])
<class 'numpy.ndarray'>
(3, 512)
[tensor(0.9922), tensor(0.9922)]
<class 'int'>
74714


In [5]:
args = Args(act_mode_8bit=True)

train_set, test_set = smsmotordata.smsmotordata_dur_0_5_overlap_0_75_dbu_ind_get_datasets_for_eval_for_analysis((data_path, args), load_train=True, load_test=True)


# Choose to have label 0
selected_idx = 45
sample_item = test_set[selected_idx]
sample_input = sample_item[0].cpu().detach().numpy().astype(np.int64)
sample_label = sample_item[1]


Pickle file path: /data2/ml/SMSMotorData/processed/train_anomaly_label_True_ds_ratio_2_signal_dur_0.5_overlap_ratio_0.75_dc_removed_False_dbu_ind_True_ds_after_fft_False_norm_per_ch_False.pkl ...


Generating dataset pickle files from the raw data files...

Train dataset length: 20160


Pickle file path: /data2/ml/SMSMotorData/processed/test_anomaly_label_True_ds_ratio_2_signal_dur_0.5_overlap_ratio_0.75_dc_removed_False_dbu_ind_True_ds_after_fft_False_norm_per_ch_False.pkl ...


Generating dataset pickle files from the raw data files...

Test dataset length: 30240



In [6]:
len(test_set)

30240

In [9]:
from torch.utils import data
batch_size = 8
test_dataloader = data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

for idx, (signal, label, file_idx) in enumerate(test_dataloader):
    print(signal.shape)
    print(signal.dtype)
    print([signal.min(), signal.max()])

    print(label.shape)
    print(label.dtype)
    print([label.min(), label.max()])

    print(file_idx.dtype)
    print(file_idx)
    break


torch.Size([8, 3, 512])
torch.float32
[tensor(127.), tensor(127.)]
torch.Size([8, 1])
torch.int64
[tensor(0), tensor(1)]
torch.int64
tensor([75555, 74755, 74549, 71823, 87064, 88103, 75185, 74601])


In [10]:
train_set, test_set = smsmotordata.smsmotordata_dur_0_5_overlap_0_75_dbu_ind_get_datasets_for_train((data_path, args), load_train=True, load_test=True)


Pickle file path: /data2/ml/SMSMotorData/processed/train_anomaly_label_False_ds_ratio_2_signal_dur_0.5_overlap_ratio_0.75_dc_removed_False_dbu_ind_True_ds_after_fft_False_norm_per_ch_False.pkl ...


Pickle files are already generated ...

Train dataset length: 20160


Pickle file path: /data2/ml/SMSMotorData/processed/test_anomaly_label_False_ds_ratio_2_signal_dur_0.5_overlap_ratio_0.75_dc_removed_False_dbu_ind_True_ds_after_fft_False_norm_per_ch_False.pkl ...


Pickle files are already generated ...

Test dataset length: 5040



In [11]:
from torch.utils import data
batch_size = 8
test_dataloader = data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

for idx, (signal, label) in enumerate(test_dataloader):
    print(signal.shape)
    print(signal.dtype)
    print([signal.min(), signal.max()])

    print(label.shape)
    print(label.dtype)
    print([label.min(), label.max()])

    break

torch.Size([8, 3, 512])
torch.float32
[tensor(127.), tensor(127.)]
torch.Size([8, 3, 512])
torch.float32
[tensor(127.), tensor(127.)]
